set up

In [1]:
import os
import sys
import json
import psycopg2
import configparser

In [2]:
# Load config file
config = configparser.ConfigParser()
config.read('../config/config.conf')

# Load configuration file (json format) for source database 
with open(config['path']['config.sv']) as json_file:
    source = json.load(json_file)

DATA_DIR = config['path']['source']
HOST = config['database']['host.sv']
PORT = config['database']['port.sv']


path = config['path']
temp_dir = f"..{path['temp']}"

sys.path.append(config['path']['root'])

from utils import (insert as insert, loggs, parse as parse, process, validate as validate)

transform

In [7]:
from importlib import reload
reload(validate)
reload(loggs)
reload(parse)
reload(process)
reload(insert)


# Connect to Postgres database
cnx = psycopg2.connect(
    database=source['database'],
    user=source['user'],
    password=source['password'],
    host=source.get('host', 'localhost'),  # 'localhost' as default
    port=source.get('port', '5432')  # '5432' is postgres' default
)


# Usage
temp_dir = f"..{path['temp']}"
for filename in os.listdir(temp_dir):
    print(filename)
    if filename.endswith('.txt'):
        full_path = os.path.join(temp_dir, filename)
        process.process_file(full_path, cnx)
        # os.remove(full_path)


# Close Postgres connection
cnx.close()

report_7.txt
Validation failed: Layout validation failed for row ['lucyrojo63@yahoo.com', '', '', '', '08/02/2013 18:30', '09/02/2013 09:55', '2', '0', '09/02/2013 09:55', '2', '0', '"3', '3"', '201.141.137.184', 'Chrome Generic', 'WinXP']
Validation failed: Layout validation failed for row ['lynbar777@yahoo.com', '', '', 'SI', '08/02/2013 18:30', '08/02/2013 12:20', '1', '0', '08/02/2013 12:22', '2', '0', '"3', '3"', '187.245.156.195', 'Firefox 3.6', 'MacOSX']
Validation failed: Layout validation failed for row ['lupitadecena@live.com.mx', '', '', '', '08/02/2013 18:30', '08/02/2013 12:19', '2', '0', '08/02/2013 12:20', '2', '0', '"0', '3"', '189.254.205.150', 'IE 9.0', 'Win7']
Validation failed: Layout validation failed for row ['chinchuma@yahoo.com', '', '', '', '08/02/2013 18:30', '20/02/2013 17:48', '2', '0', '20/02/2013 17:48', '2', '0', '"0', '3"', '190.51.32.134', '', 'unknown']
Validation failed: Layout validation failed for row ['samanta.llaguno@algo.com', '', '', '', '08/02/